In [3]:
%run environnement.ipynb

Random seed set to 314
Tensorflow framework: GPU is available
(Index(['hour', 'target', 'text', 'tokenizer with lowercase',
       'tokenizer with lowercase, handle stripping, and length reduction',
       'tokenizer with lowercase and alpha',
       'tokenizer with lowercase, alpha and emoji',
       'tokenizer with lowercase, alpha, and no stop words',
       'tokenizer with lowercase, alpha and emoji, and no stop words'],
      dtype='object'), array([2, 0, 1, 3, 4, 5, 6, 7, 8]))
<class 'pandas.core.frame.DataFrame'>
Index: 1596630 entries, 0 to 799999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1596630 non-null  object
 1   target  1596630 non-null  int8  
dtypes: int8(1), object(1)
memory usage: 25.9+ MB
None
(12772,) (3194,) (12772,) (3194,)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer

# from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import accuracy_score

In [5]:
# Define the URI of the MLflow server and the name of the experiment
experiment = "ml_tfidf"
run = "baseline"

# Args for dataset preparation
col_name = "tokenizer with lowercase, alpha and emoji, and no stop words"
val_split = 0.2

# Args for the model
ngram_range = (1, 1)
max_features = 1000

In [6]:
# Set the tracking URI
mlflow.set_tracking_uri(URI)
# try to connect to the server
try:
    mlflow.tracking.get_tracking_uri()
except Exception as e:
    print(f"Cannot connect to the server : {URI}. Check the server status.")
    raise e
# Set, and create if necessary, the experiment
try:
    mlflow.create_experiment(experiment)
except:
    pass
mlflow.set_experiment(experiment)

<Experiment: artifact_location='/home/hedredo/github/p7/mlruns/1', creation_time=1735481304954, experiment_id='1', last_update_time=1735481304954, lifecycle_stage='active', name='ml_tfidf', tags={}>

In [7]:
from sklearn.pipeline import Pipeline

In [8]:
def cross_score(X_train, y_train, model, col, cv=10):
    """
    Perform a cross-validation on the model
    :param X_train: Training data
    :param y_train: Training target
    :param model: Model to train
    :param cv: Number of folds
    :return: scores
    """
    # Perform a cross-validation
    scores = cross_validate(model, X_train, y_train, cv=cv, scoring=["accuracy", "f1"])
    # Return the mean scores
    return pd.DataFrame(scores).mean().rename(col)

In [11]:
# create a pipeline with Tf-Idf and Logistic Regression
model = Pipeline(
    [
        (
            "tfidf",
            TfidfVectorizer(
                ngram_range=(1, 1),
                max_features=500,
                strip_accents="unicode",
            ),
        ),
        ("logistic", LogisticRegression(max_iter=1000)),
    ]
)

In [13]:
X_train, X_test, y_train, y_test = load_splits_from_parquet(
    X_train, X_test, y_train, y_test, col_name, PATH_PARQUET
)
mlflow.sklearn.autolog()
with mlflow.start_run() as active_run:
    # Load the file
    # create a pipeline with Tf-Idf and Logistic Regression
    model = Pipeline(
        [
            (
                "tfidf",
                TfidfVectorizer(
                    ngram_range=(1, 1),
                    max_features=500,
                    strip_accents="unicode",
                ),
            ),
            ("logistic", LogisticRegression(max_iter=1000)),
        ]
    )
    # Fit the model
    model.fit(X_train, y_train)
    # Predict the target
    accuracy = model.score(X_test, y_test)
    y_pred = model.predict(X_test)
    model_uri = f"runs:/{active_run.info.run_id}/model"

2024/12/29 15:13:58 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.
2024/12/29 15:14:01 WARNING mlflow.sklearn: Unrecognized dataset type <class 'pandas.core.series.Series'>. Dataset logging skipped.
2024/12/29 15:14:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-bug-185 at: http://localhost:5000/#/experiments/1/runs/946de5cfda4b44cca67335e538b1987c.
2024/12/29 15:14:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/1.
